<a href="https://colab.research.google.com/github/leartssy/BlenderDiffuseNST/blob/main/DiffuseST_03_Tileability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer by DiffuseST #

# Initial Setup

Downgrade to python 3.11 as that is what Blender uses

In [ ]:
!sudo apt-get update
!sudo apt-get install python3.11 python3.11-venv
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!python3 --version

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [60.9 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,290 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 http://security.ubuntu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.11.13


Clone the main repo

In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/

/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3


In [3]:
!rm -rf BlenderDiffuseNST
!git clone https://github.com/leartssy/BlenderDiffuseNST.git

Cloning into 'BlenderDiffuseNST'...
remote: Enumerating objects: 4912, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 4912 (delta 38), reused 21 (delta 3), pack-reused 4818 (from 2)
Receiving objects: 100% (4912/4912), 411.68 MiB | 13.90 MiB/s, done.
Resolving deltas: 100% (89/89), done.


Setup libraries and dependencies

In [ ]:
#!pip uninstall -y diffusers transformers accelerate peft huggingface-hub tokenizers

Found existing installation: diffusers 0.34.0
Uninstalling diffusers-0.34.0:
  Successfully uninstalled diffusers-0.34.0
Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
Found existing installation: peft 0.16.0
Uninstalling peft-0.16.0:
  Successfully uninstalled peft-0.16.0
Found existing installation: huggingface-hub 0.33.4
Uninstalling huggingface-hub-0.33.4:
  Successfully uninstalled huggingface-hub-0.33.4
Found existing installation: tokenizers 0.21.2
Uninstalling tokenizers-0.21.2:
  Successfully uninstalled tokenizers-0.21.2


In [3]:
%cd BlenderDiffuseNST/DiffuseST_with_TexTile/
!pip install -r requirements.txt
%cd ..

/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/DiffuseST_with_TexTile
/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST


Restart Runtime if in colab and do install requirements again, then continue

In [4]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Using CPU.")

CUDA is available! Device name: Tesla T4


Load the blipdiffusion model

In [5]:
!git clone https://huggingface.co/salesforce/blipdiffusion

Cloning into 'blipdiffusion'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 28 (from 1)
Unpacking objects: 100% (32/32), 520.37 KiB | 792.00 KiB/s, done.
Filtering content: 100% (6/6), 8.24 GiB | 66.21 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/blipdiffusion/.git/hooks/post-checkout': Permission denied


In [6]:
%cd ..

/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3


In [15]:
%cd blipdiffusion/

/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/blipdiffusion


In [16]:
# The 'blipdiffusion' repository was cloned to:
cloned_model_key = !pwd
cloned_model_key = str(cloned_model_key[0])
print(f"The model was cloned to {cloned_model_key}")

The model was cloned to /content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/blipdiffusion


Convert saved model.bin files to safetensors - because of version compatibility

In [17]:
from safetensors.torch import save_file
import torch
%cd "unet"
# Load the PyTorch .bin model
state_dict = torch.load("diffusion_pytorch_model.bin", map_location="cpu")

# Save as .safetensors
save_file(state_dict, "diffusion_pytorch_model.safetensors")

/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/blipdiffusion/unet


In [18]:
%cd ..
%cd "vae"
# Load the PyTorch .bin model
state_dict = torch.load("diffusion_pytorch_model.bin", map_location="cpu")

# Save as .safetensors in the same folder
save_file(state_dict, "diffusion_pytorch_model.safetensors")

/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/blipdiffusion
/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/blipdiffusion/vae


In [20]:
%cd ..
%cd ..

/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST


Install TexTile

In [21]:
!pip install textile-metric

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.3 MB/s eta 0:00:00
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12065 sha256=e29bc9eba5b18a6fdbddd35c7f10aa4acdeafa4e0da2c9fd11b1935b8dcefe0c
  Stored in directory: /root/.cache/pip/wheels/a5/4d/c7/f3cf0f75c746c219090060131fe00f1523cc2c5484991f4030
Successfully built progressbar


In [25]:
%cd DiffuseST_with_TexTile

/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/DiffuseST_with_TexTile


In [26]:
#initialize the textile loss
import textile
from textile.utils.image_utils import read_and_process_image

loss_textile = textile.Textile()

Model not found, downloading pretrained model:


#Run normal Diffusion Style Transfer

Choose the content and style image directory as well as the output path

In [27]:
#content_path = str(input("Enter the content images folder"))
#style_path = str(input("Enter the style images folder"))
output_path = str(input("Enter the output path folder"))

Enter the output path folder/content/drive/MyDrive/Colab Notebooks/DiffusionST_v3/BlenderDiffuseNST/DiffuseST_with_TexTile/output


In [ ]:
#display all option for run.py
!python run.py --help

2025-11-26 15:54:35.034129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764172475.054139    6808 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764172475.060184    6808 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: run.py [-h] [--content_path CONTENT_PATH] [--style_path STYLE_PATH]
              [--output_dir OUTPUT_DIR] [--sd_version {1.5,2.0,2.1}]
              [--device DEVICE] [--alpha ALPHA] [--seed SEED]
              [--ddpm_steps DDPM_STEPS] [--steps_to_save STEPS_TO_SAVE]
              [--ddim_steps DDIM_STEPS]
              [--textile_guidance_scale TEXTILE_GUIDANCE_SCALE]
              [--inversion_prompt INVERSION_PROMPT] [--

Run diffusion style transfer: put content images in images/content and style images in images/style

In [ ]:
#diffusion style transfer without tileability = texture_guidance_scale 0.0
!python run.py --textile_guidance_scale 0.0 --alpha 0.6 --prefix_name "StyleAlpha: 0.6" --model_key {cloned_model_key}

# Adding Tileability by TexTile #

Textile_guidance_scale values:
1.0-10.0: low
10.0-20.0: medium
20.0+:high

In [ ]:
!python run.py --model_key {cloned_model_key} --content_path {content_path} --style_path {style_path} --output_dir {output_path} --textile_guidance_scale 25.0 --alpha 0.6 --ddim_steps 50 --prefix_name "Tileable_"

python3: can't open file '/content/run.py': [Errno 2] No such file or directory


In [ ]:
#optional: push the other scripts and folders also to git
# Replace YourRepo with the actual name of your repository folder
%cd /content/BlenderDiffuseNST

# 1. Stage all new, modified, and deleted files for the next commit
!git add .

# 2. Create the local snapshot (commit) with a descriptive message
!git commit -m "Added output folder"

# 3. Send the local commit(s) to your GitHub repository
!git push

